In [56]:
import os
import pandas as pd
from torch.utils.data import DataLoader
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms.functional as tx
import torchvision.transforms as T
import numpy as np
from torch import nn
from torchmetrics import Accuracy
from PIL import Image
import random

In [57]:
few_data = False
data_num = 100
batch_size = 10

In [58]:
class_map=['A10', 'A400M', 'AG600', 'AV8B', 'B1', 'B2', 'B52', 'Be200', 'C130', 'C17', 'C5', 'E2', 'EF2000', 'F117', 'F14', 'F15', 'F16', 'F18', 'F22', 'F35', 'F4', 'J20', 'JAS39', 'Mig31', 'Mirage2000', 'MQ9', 'Rafale', 'RQ4', 'SR71', 'Su34', 'Su57', 'Tornado', 'Tu160', 'Tu95', 'U2', 'US2', 'V22', 'Vulcan', 'XB70', 'YF23']

In [59]:
global_mean = [120.74446532507794, 129.82000781821228, 138.8076694793008]
global_std = [51.8159762582478, 51.42346227240079, 52.72563864069662]

In [60]:
class MyDataset():
    def __init__(self, csv_file, augment=False):
        if few_data:
            self.df = pd.read_csv(csv_file).iloc[:data_num]
        else:
            self.df = pd.read_csv(csv_file)
        self.image_size=256
        self.mean = np.array(global_mean, dtype=np.float32)
        self.std = np.array(global_std, dtype=np.float32)
        self.augment = augment
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df["type"][idx]
        image_name = self.df["name"][idx]
#         path = 
        image = cv2.imread("../data/militaryaircraftdetectiondataset/crop/"+class_map[label]+"/"+image_name)
        image = cv2.resize(image, dsize=(self.image_size, self.image_size), interpolation=cv2.INTER_LINEAR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        if self.augment:
            image = augmentation(image)
        image = tx.to_tensor(image)
        image = tx.resize(image, (self.image_size, self.image_size))
        image = tx.normalize(image, self.mean/255, self.std/255) 
#         image = tx.to_tensor(image)
        return image, label

In [61]:
test_dataset = MyDataset("../data/test_data")
test_dataloader = DataLoader(test_dataset,batch_size=batch_size)

In [62]:
image, label = next(iter(test_dataloader))
print(len(test_dataset))

1387


In [83]:
model = torchvision.models.efficientnet_b6(pretrained=True)
model.classifier = torch.nn.Linear(2304, 40, bias=True)
model.load_state_dict(torch.load("../data/models/best.pth"))

<All keys matched successfully>

In [84]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
loss_fn = nn.CrossEntropyLoss()
model.to(device)
print(device)

cuda


In [85]:
def get_score(pred, label):
    pred = torch.argmax(pred, axis = 1)
    score = 0
    for i in range(len(pred)):
        if pred[i]==label[i]:
            score+=1

    return score

In [86]:
def test(dataloader, loss_fn,model):
    model.eval()
    total_loss = 0
    total_acc = 0
    for i, (image, label) in enumerate(dataloader):
        image = image.to(device)
        label = label.to(device)
        pred = model(image)
        acc = get_score(pred, label)
        loss = loss_fn(pred, label)
        total_loss += loss.item()
        total_acc += acc
        if i%200==0:
            print("test average total loss : ", total_loss/(i+1))
    return total_loss, total_acc

In [87]:
test_loss, test_acc = test(test_dataloader, loss_fn, model)
print(test_loss/len(test_dataloader), test_acc/len(test_dataset))




test average total loss :  0.00039845696301199496
0.4494945693635783 0.9329488103821196


In [13]:
!nvidia-smi

Fri Dec 16 18:26:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.25       Driver Version: 522.25       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 73%   80C    P2   297W / 350W |  20575MiB / 24576MiB |     64%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------